In [14]:
from dataclasses import dataclass, field
import functools
import itertools
import re

from pyprojroot import here

In [24]:
@dataclass
class Volume:
    cubes: set[tuple[int, int, int]]

    def cubeSurfaceArea(self, cube: tuple[int, int, int]):
        x, y, z = cube
        adjacentCubes = [(x, y, z - 1),(x, y, z + 1), (x, y - 1, z), (x, y + 1, z),(x - 1, y, z), (x + 1, y, z)]
        return sum([cube not in self.cubes for cube in adjacentCubes])

    def surfaceArea(self) -> int:
        return sum(map(self.cubeSurfaceArea, self.cubes))

In [27]:
@dataclass
class Boulder(Volume):
    container: set[tuple[int, int, int]] = field(init=False)

    def __post_init__(self):
        self.container = set(itertools.product(
            range(min(x for x, y, z in self.cubes) - 1, max(x for x, y, z in self.cubes) + 2),
            range(min(y for x, y, z in self.cubes) - 1, max(y for x, y, z in self.cubes) + 2),
            range(min(z for x, y, z in self.cubes) - 1, max(z for x, y, z in self.cubes) + 2)
        ))

    def findNegativeSpace(self) -> set[tuple[int, int, int]]:
        negativeSpace = set()
        stack = [sorted(self.container)[0]]

        while stack:
            x, y, z = stack.pop()
            negativeSpace.add((x, y, z))
            adjacentCubes = [(x, y, z - 1),(x, y, z + 1), (x, y - 1, z), (x, y + 1, z),(x - 1, y, z), (x + 1, y, z)]
            children = [cube for cube in adjacentCubes if (cube in self.container) and (cube not in negativeSpace) and (cube not in self.cubes)]
            stack.extend(children)
            
        return negativeSpace

    def findAirPockets(self) -> Volume:
        return Volume(self.container - self.findNegativeSpace() - self.cubes)

In [39]:
testPath = here('./18/test.txt')
path = here('./18/input.txt')

with open(path, 'r') as fp:
    lines = fp.readlines()
    cubes = set([tuple(int(char) for char in (re.findall('\d+', line))) for line in lines])
    boulder = Boulder(cubes)

In [40]:
boulder.surfaceArea()

4308

In [41]:
boulder.findAirPockets().surfaceArea()

1768

In [42]:
boulder.surfaceArea() - boulder.findAirPockets().surfaceArea()

2540